In [1]:
from fastai.text.all import *

In [2]:
torch.cuda.is_available()

True

In [2]:
path = untar_data(URLs.IMDB)
path.ls()

(#7) [Path('/home/lazylearner/.fastai/data/imdb/unsup'),Path('/home/lazylearner/.fastai/data/imdb/tmp_clas'),Path('/home/lazylearner/.fastai/data/imdb/train'),Path('/home/lazylearner/.fastai/data/imdb/imdb.vocab'),Path('/home/lazylearner/.fastai/data/imdb/README'),Path('/home/lazylearner/.fastai/data/imdb/tmp_lm'),Path('/home/lazylearner/.fastai/data/imdb/test')]

In [3]:
(path/'train').ls()

(#4) [Path('/home/lazylearner/.fastai/data/imdb/train/pos'),Path('/home/lazylearner/.fastai/data/imdb/train/unsupBow.feat'),Path('/home/lazylearner/.fastai/data/imdb/train/labeledBow.feat'),Path('/home/lazylearner/.fastai/data/imdb/train/neg')]

#### Fine-tuning a language model on IMDb

In [5]:
path

Path('/home/lazylearner/.fastai/data/imdb')

In [3]:
path = Path('/home/lazylearner/.fastai/data/imdb')

In [4]:
BATCH_SIZE = 32
dls_lm = TextDataLoaders.from_folder(path, is_lm=True, bs=BATCH_SIZE, valid_pct=0.1)

In [5]:
dls_lm.show_batch(max_n=5)

,text,text_
0,"xxbos i saw this movie when i was about 12 when it came out . i recall the scariest scene was the big bird eating men dangling helplessly from parachutes right out of the air . xxmaj the horror . xxmaj the horror . \n\n xxmaj as a young kid going to these cheesy b films on xxmaj saturday afternoons , i still was tired of the formula for these monster type","i saw this movie when i was about 12 when it came out . i recall the scariest scene was the big bird eating men dangling helplessly from parachutes right out of the air . xxmaj the horror . xxmaj the horror . \n\n xxmaj as a young kid going to these cheesy b films on xxmaj saturday afternoons , i still was tired of the formula for these monster type movies"
1,"( jon xxmaj voight ) , xxmaj lewis ( burt xxmaj reynolds ) , xxmaj drew ( ronny xxmaj cox ) and xxmaj bobby ( ned xxmaj beatty ) , decide to go on a rafting trip on the xxmaj cahulawassee river , before it is flooded . \n\n xxmaj they wanted to have fun , to have a nice weekend in the nature . \n\n xxmaj but when two mountain men","jon xxmaj voight ) , xxmaj lewis ( burt xxmaj reynolds ) , xxmaj drew ( ronny xxmaj cox ) and xxmaj bobby ( ned xxmaj beatty ) , decide to go on a rafting trip on the xxmaj cahulawassee river , before it is flooded . \n\n xxmaj they wanted to have fun , to have a nice weekend in the nature . \n\n xxmaj but when two mountain men cross"
2,"the women have a few drinks and share their sexual encounters with each other . xxmaj its much more personal and people can relate to it . xxmaj its much more engaging and emotional on a new level than other dramas focusing on women and their lives like "" sex and the xxmaj city , xxmaj lipstick xxmaj jungle … . "" \n\n xxmaj dr . xxmaj katie xxmaj xxunk , is","women have a few drinks and share their sexual encounters with each other . xxmaj its much more personal and people can relate to it . xxmaj its much more engaging and emotional on a new level than other dramas focusing on women and their lives like "" sex and the xxmaj city , xxmaj lipstick xxmaj jungle … . "" \n\n xxmaj dr . xxmaj katie xxmaj xxunk , is a"
3,"2004 as a part of a 5 disc / 10 movies boxed - set named "" sci - fi - 10 xxup great xxup films ! "" . \n\n r xxrep 8 i ght … .. xxbos xxmaj on xxmaj june 14 1905 , during the xxmaj russian xxmaj revolution of that year , sailors aboard the xxmaj russian battleship xxmaj potemkin rebelled against their oppressive officers . xxmaj frustrated with the","as a part of a 5 disc / 10 movies boxed - set named "" sci - fi - 10 xxup great xxup films ! "" . \n\n r xxrep 8 i ght … .. xxbos xxmaj on xxmaj june 14 1905 , during the xxmaj russian xxmaj revolution of that year , sailors aboard the xxmaj russian battleship xxmaj potemkin rebelled against their oppressive officers . xxmaj frustrated with the second"
4,"wagons of immigrants riding in circles around the encampment of regulations ( early westerns to the contrary this was a film making device and not an actual tactic of the xxmaj indians ) . xxmaj and weeks prior to the arrival of the regulators a number of xxmaj johnson xxmaj county residents were hanged without trial including xxmaj jim xxmaj xxunk , the keeper of a modest road ranch , and his","of immigrants riding in circles around the encampment of regulations ( early westerns to the contrary this was a film making device and not an actual tactic of the xxmaj indians ) . xxmaj and weeks prior to the arrival of the regulators a number of xxmaj johnson xxmaj county residents were hanged without trial including xxmaj jim xxmaj xxunk , the keeper of a modest road ranch , and his wife"


In [6]:
dls_lm.bs

32

In [ ]:
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()

In [8]:
learn.model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(60008, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(60008, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=60008, bias=True)
    (output_dp): RNNDropout()
  )
)

In [9]:
TEXT = "The color of the sky is"
N_WORDS = 40
N_SENTENCES = 2

In [10]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

The color of the sky is a dark color , the symbolism being reflected in the spirit of the Moon . The blue sky appears to be a part of the sky , which makes the sky visible in throughout the world .
The color of the sky is similar to Earth , with the sun being white with a bright grey surface . The sky is a dark blue , whereas the sky is dark . The sky has a dark blue sky , and


In [11]:
TEXT = "I hated this movie"
N_WORDS = 30
N_SENTENCES = 2

In [12]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

i hated this movie : " i wanted to do that , i made this things i was excited about . But i wanted to bring the movie to the head of the
i hated this movie for English actor Albert Chevalier . The film was never made available at the Cannes Film Festival , France 's national film


In [11]:
?? learn.fit_one_cycle

In [9]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.468313,4.189966,0.281453,66.020561,1:10:04


In [10]:
learn.save('1epoch')

Path('/home/lazylearner/.fastai/data/imdb/models/1epoch.pth')

In [ ]:
# learn = learn.load('1epoch')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

In [ ]:
learn.save_encoder('finetuned')

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]